In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [10]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [11]:
def create_convnext_like_model(input_dim, num_classes, sliding_window_size=3, loss_function=None):
    """
    Create a ConvNeXt-like model using Conv1D layers.
    """
    input_layer = Input(shape=(input_dim,))
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)

    x = reshaped_input
    filters = [input_dim * 2, input_dim, input_dim // 2, input_dim // 4, input_dim // 8]

    for f in filters:
        f = max(16, int(f))  # Ensure minimum of 16 filters
        x = Conv1D(filters=f, kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
        x = LayerNormalization()(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

    return model

In [12]:
class HighlyAdaptiveFocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=1.0, gamma_start=2.0, num_samples=None, precision=None, name="highly_adaptive_focal_loss"):
        super().__init__(name=name)
        self.alpha = alpha
        self.gamma_start = gamma_start
        self.num_samples = num_samples
        self.precision = precision
        self.epoch = tf.Variable(1.0, trainable=False, dtype=tf.float32)  # Track epoch dynamically

    def update_epoch(self, epoch):
        """ Update the current epoch dynamically during training """
        self.epoch.assign(tf.cast(epoch, tf.float32))

    def call(self, y_true, y_pred):
        """ Compute Highly Adaptive Focal Loss """
        y_pred = tf.clip_by_value(y_pred, 1e-9, 1.0)  # Prevent log(0) issues
        pt = tf.reduce_sum(y_true * y_pred, axis=-1)  # Probability of true class

        # Compute adaptive gamma
        gamma_epoch = 1.0 / tf.sqrt(self.epoch + 1e-9)
        gamma_balance = tf.sqrt(tf.gather(self.num_samples, tf.argmax(y_true, axis=-1)) / tf.reduce_max(self.num_samples))
        gamma_precision = tf.math.log(tf.gather(self.precision, tf.argmax(y_true, axis=-1)) / tf.reduce_max(self.precision) + 1) + 0.5

        gamma_adaptation = self.gamma_start * gamma_epoch * gamma_balance * gamma_precision

        # Compute focal weight and loss
        focal_weight = (1 - pt) ** gamma_adaptation
        loss = -self.alpha * focal_weight * tf.math.log(pt)

        return tf.reduce_mean(loss)

In [13]:
def run_model(train_data_path, test_data_path, is_string_labels=False, label_mapping=None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # Compute Dataset Statistics for HAFL
    num_classes = train_y.shape[1]
    num_samples = tf.constant([np.sum(train_y[:, i]) for i in range(num_classes)], dtype=tf.float32)
    precision = tf.constant([1.0] * num_classes, dtype=tf.float32)  # Placeholder, can be updated

    # Initialize HAFL Loss
    hafl_loss = HighlyAdaptiveFocalLoss(num_samples=num_samples, precision=precision)

    # EarlyStopping Callback (even though mentions loss, it tracks HAFL as the training loss)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, mode='min')

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        model = create_convnext_like_model(train_X.shape[1], test_y.shape[1], loss_function=hafl_loss)

        print(f"Run {run + 1}/{num_runs} started...")

        # Update epoch value before training
        hafl_loss.update_epoch(run + 1)

        history = model.fit(
            train_X, train_y, 
            epochs=100, 
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [ ]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/50 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0014
test_accuracy: 0.9994
balanced_accuracy: 0.8565
roc_auc: 1.0000
pr_auc: 1.0000
f2: 0.9994
mcc: 0.9983
0_precision: 1.0000
0_recall: 0.9999
0_f1-score: 0.9999
0_support: 9117.0000
1_precision: 1.0000
1_recall: 1.0000
1_f1-score: 1.0000
1_support: 10.0000
2_precision: 0.8718
2_recall: 1.0000
2_f1-score: 0.9315
2_support: 34.0000
3_precision: 0.9994
3_recall: 0.9989
3_f1-score: 0.9992
3_support: 1781.0000
4_precision: 0.9985
4_recall: 0.9969
4_f1-score: 0.9977
4_support: 653.0000
5_precision: 0.0000
5_recall: 0.0000
5_f1-score: 0.0000
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9994
macro avg_precision: 0.8385
macro avg_recall: 0.8565
macro avg_f1-score: 0.8469
macro avg_support: 11600.0000
weighted avg_precision: 0.9993
weighted avg_recall: 0.9994
weighted avg_f1-score: 0.9993
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs:

In [ ]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_loss: 0.0994
test_accuracy: 0.8729
balanced_accuracy: 0.8192
roc_auc: 0.9874
pr_auc: 0.9492
f2: 0.8727
mcc: 0.7954
0_precision: 0.8810
0_recall: 0.8434
0_f1-score: 0.8618
0_support: 42368.0000
1_precision: 0.8752
1_recall: 0.9083
1_f1-score: 0.8915
1_support: 56661.0000
2_precision: 0.8667
2_recall: 0.8754
2_f1-score: 0.8710
2_support: 7151.0000
3_precision: 0.7962
3_recall: 0.7614
3_f1-score: 0.7784
3_support: 549.0000
4_precision: 0.7676
4_recall: 0.7114
4_f1-score: 0.7385
4_support: 1899.0000
5_precision: 0.7473
5_recall: 0.7961
5_f1-score: 0.7709
5_support: 3473.0000
6_precision: 0.9471
6_recall: 0.8384
6_f1-score: 0.8894
6_support: 4102.0000
accuracy: 0.8729
macro avg_precision: 0.8402
macro avg_recall: 0.8192
macro avg_f1-score: 0.8288
macro avg_support: 116203.0000
weighted avg_precision: 0.8734
weighted avg_recall: 0.8729
weighted avg_f1-score: 0.8727
weighted avg_support: 116203.0000
Run 2/50 started...



Aggregated Metrics for 2 runs:
test_loss: 0.2086
test_accuracy: 0.8002
balanced_accuracy: 0.6623
roc_auc: 0.9646
pr_auc: 0.8683
f2: 0.7981
mcc: 0.6729
0_precision: 0.8079
0_recall: 0.7747
0_f1-score: 0.7910
0_support: 42368.0000
1_precision: 0.8062
1_recall: 0.8625
1_f1-score: 0.8332
1_support: 56661.0000
2_precision: 0.7606
2_recall: 0.7693
2_f1-score: 0.7650
2_support: 7151.0000
3_precision: 0.6779
3_recall: 0.5601
3_f1-score: 0.6078
3_support: 549.0000
4_precision: 0.6177
4_recall: 0.4834
4_f1-score: 0.5344
4_support: 1899.0000
5_precision: 0.6134
5_recall: 0.4823
5_f1-score: 0.5101
5_support: 3473.0000
6_precision: 0.8789
6_recall: 0.7038
6_f1-score: 0.7791
6_support: 4102.0000
accuracy: 0.8002
macro avg_precision: 0.7375
macro avg_recall: 0.6623
macro avg_f1-score: 0.6887
macro avg_support: 116203.0000
weighted avg_precision: 0.7972
weighted avg_recall: 0.8002
weighted avg_f1-score: 0.7961
weighted avg_support: 116203.0000
Run 3/50 started...



Aggregated Metrics for 3 runs:
test_loss: 0.2541
test_accuracy: 0.7795
balanced_accuracy: 0.5959
roc_auc: 0.9578
pr_auc: 0.8453
f2: 0.7763
mcc: 0.6396
0_precision: 0.7747
0_recall: 0.7791
0_f1-score: 0.7759
0_support: 42368.0000
1_precision: 0.7960
1_recall: 0.8311
1_f1-score: 0.8128
1_support: 56661.0000
2_precision: 0.7306
2_recall: 0.7837
2_f1-score: 0.7548
2_support: 7151.0000
3_precision: 0.6921
3_recall: 0.4032
3_f1-score: 0.4582
3_support: 549.0000
4_precision: 0.7135
4_recall: 0.3407
4_f1-score: 0.3910
4_support: 1899.0000
5_precision: 0.5476
5_recall: 0.3623
5_f1-score: 0.4031
5_support: 3473.0000
6_precision: 0.8584
6_recall: 0.6715
6_f1-score: 0.7516
6_support: 4102.0000
accuracy: 0.7795
macro avg_precision: 0.7304
macro avg_recall: 0.5959
macro avg_f1-score: 0.6211
macro avg_support: 116203.0000
weighted avg_precision: 0.7771
weighted avg_recall: 0.7795
weighted avg_f1-score: 0.7728
weighted avg_support: 116203.0000
Run 4/50 started...



Aggregated Metrics for 4 runs:
test_loss: 0.2893
test_accuracy: 0.7621
balanced_accuracy: 0.5647
roc_auc: 0.9536
pr_auc: 0.8316
f2: 0.7563
mcc: 0.6138
0_precision: 0.7864
0_recall: 0.7084
0_f1-score: 0.7366
0_support: 42368.0000
1_precision: 0.7682
1_recall: 0.8522
1_f1-score: 0.8054
1_support: 56661.0000
2_precision: 0.7127
2_recall: 0.7950
2_f1-score: 0.7497
2_support: 7151.0000
3_precision: 0.7202
3_recall: 0.3192
3_f1-score: 0.3747
3_support: 549.0000
4_precision: 0.6419
4_recall: 0.2913
4_f1-score: 0.3469
4_support: 1899.0000
5_precision: 0.5208
5_recall: 0.3226
5_f1-score: 0.3719
5_support: 3473.0000
6_precision: 0.8282
6_recall: 0.6645
6_f1-score: 0.7356
6_support: 4102.0000
accuracy: 0.7621
macro avg_precision: 0.7112
macro avg_recall: 0.5647
macro avg_f1-score: 0.5887
macro avg_support: 116203.0000
weighted avg_precision: 0.7638
weighted avg_recall: 0.7621
weighted avg_f1-score: 0.7520
weighted avg_support: 116203.0000
Run 5/50 started...



Aggregated Metrics for 5 runs:
test_loss: 0.3138
test_accuracy: 0.7530
balanced_accuracy: 0.5493
roc_auc: 0.9520
pr_auc: 0.8260
f2: 0.7455
mcc: 0.6007
0_precision: 0.7968
0_recall: 0.6677
0_f1-score: 0.7154
0_support: 42368.0000
1_precision: 0.7517
1_recall: 0.8661
1_f1-score: 0.8017
1_support: 56661.0000
2_precision: 0.7041
2_recall: 0.7966
2_f1-score: 0.7458
2_support: 7151.0000
3_precision: 0.6962
3_recall: 0.2903
3_f1-score: 0.3539
3_support: 549.0000
4_precision: 0.6653
4_recall: 0.2556
4_f1-score: 0.3168
4_support: 1899.0000
5_precision: 0.5081
5_recall: 0.2882
5_f1-score: 0.3428
5_support: 3473.0000
6_precision: 0.8009
6_recall: 0.6805
6_f1-score: 0.7319
6_support: 4102.0000
accuracy: 0.7530
macro avg_precision: 0.7033
macro avg_recall: 0.5493
macro avg_f1-score: 0.5726
macro avg_support: 116203.0000
weighted avg_precision: 0.7580
weighted avg_recall: 0.7530
weighted avg_f1-score: 0.7405
weighted avg_support: 116203.0000
Run 6/50 started...



Aggregated Metrics for 6 runs:
test_loss: 0.3348
test_accuracy: 0.7437
balanced_accuracy: 0.5376
roc_auc: 0.9506
pr_auc: 0.8213
f2: 0.7359
mcc: 0.5902
0_precision: 0.7667
0_recall: 0.7049
0_f1-score: 0.7176
0_support: 42368.0000
1_precision: 0.7639
1_recall: 0.8219
1_f1-score: 0.7839
1_support: 56661.0000
2_precision: 0.6959
2_recall: 0.8059
2_f1-score: 0.7450
2_support: 7151.0000
3_precision: 0.6968
3_recall: 0.2738
3_f1-score: 0.3450
3_support: 549.0000
4_precision: 0.6892
4_recall: 0.2256
4_f1-score: 0.2871
4_support: 1899.0000
5_precision: 0.5074
5_recall: 0.2780
5_f1-score: 0.3379
5_support: 3473.0000
6_precision: 0.8096
6_recall: 0.6531
6_f1-score: 0.7171
6_support: 4102.0000
accuracy: 0.7437
macro avg_precision: 0.7042
macro avg_recall: 0.5376
macro avg_f1-score: 0.5620
macro avg_support: 116203.0000
weighted avg_precision: 0.7531
weighted avg_recall: 0.7437
weighted avg_f1-score: 0.7314
weighted avg_support: 116203.0000
Run 7/50 started...



Aggregated Metrics for 7 runs:
test_loss: 0.3472
test_accuracy: 0.7401
balanced_accuracy: 0.5400
roc_auc: 0.9500
pr_auc: 0.8192
f2: 0.7327
mcc: 0.5860
0_precision: 0.7509
0_recall: 0.7257
0_f1-score: 0.7209
0_support: 42368.0000
1_precision: 0.7712
1_recall: 0.7997
1_f1-score: 0.7767
1_support: 56661.0000
2_precision: 0.6905
2_recall: 0.8099
2_f1-score: 0.7436
2_support: 7151.0000
3_precision: 0.6674
3_recall: 0.3086
3_f1-score: 0.3677
3_support: 549.0000
4_precision: 0.6480
4_recall: 0.2292
4_f1-score: 0.2901
4_support: 1899.0000
5_precision: 0.5114
5_recall: 0.2666
5_f1-score: 0.3310
5_support: 3473.0000
6_precision: 0.8104
6_recall: 0.6405
6_f1-score: 0.7100
6_support: 4102.0000
accuracy: 0.7401
macro avg_precision: 0.6928
macro avg_recall: 0.5400
macro avg_f1-score: 0.5629
macro avg_support: 116203.0000
weighted avg_precision: 0.7500
weighted avg_recall: 0.7401
weighted avg_f1-score: 0.7288
weighted avg_support: 116203.0000
Run 8/50 started...



Aggregated Metrics for 8 runs:
test_loss: 0.3610
test_accuracy: 0.7360
balanced_accuracy: 0.5392
roc_auc: 0.9489
pr_auc: 0.8156
f2: 0.7285
mcc: 0.5812
0_precision: 0.7365
0_recall: 0.7431
0_f1-score: 0.7224
0_support: 42368.0000
1_precision: 0.7773
1_recall: 0.7785
1_f1-score: 0.7687
1_support: 56661.0000
2_precision: 0.6848
2_recall: 0.8143
2_f1-score: 0.7422
2_support: 7151.0000
3_precision: 0.6528
3_recall: 0.3270
3_f1-score: 0.3841
3_support: 549.0000
4_precision: 0.6545
4_recall: 0.2127
4_f1-score: 0.2751
4_support: 1899.0000
5_precision: 0.5061
5_recall: 0.2640
5_f1-score: 0.3299
5_support: 3473.0000
6_precision: 0.8067
6_recall: 0.6349
6_f1-score: 0.7057
6_support: 4102.0000
accuracy: 0.7360
macro avg_precision: 0.6884
macro avg_recall: 0.5392
macro avg_f1-score: 0.5612
macro avg_support: 116203.0000
weighted avg_precision: 0.7471
weighted avg_recall: 0.7360
weighted avg_f1-score: 0.7249
weighted avg_support: 116203.0000
Run 9/50 started...


In [ ]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_train.csv", "kdd_test.csv", is_string_labels = True, label_mapping=labels_map)

In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_loss: 0.0030
test_accuracy: 0.9989
balanced_accuracy: 0.9728
roc_auc: 0.9999
pr_auc: 0.9998
f2: 0.9989
mcc: 0.9961
0_precision: 0.9996
0_recall: 0.9999
0_f1-score: 0.9998
0_support: 26862.0000
1_precision: 0.9992
1_recall: 0.9985
1_f1-score: 0.9989
1_support: 2657.0000
2_precision: 0.9956
2_recall: 0.9956
2_f1-score: 0.9956
2_support: 908.0000
3_precision: 0.9905
3_recall: 0.9943
3_f1-score: 0.9924
3_support: 522.0000
4_precision: 0.9897
4_recall: 0.9863
4_f1-score: 0.9880
4_support: 293.0000
5_precision: 0.9888
5_recall: 0.9851
5_f1-score: 0.9870
5_support: 269.0000
6_precision: 0.9739
6_recall: 0.9655
6_f1-score: 0.9697
6_support: 116.0000
7_precision: 0.9362
7_recall: 0.8302
7_f1-score: 0.8800
7_support: 53.0000
8_precision: 0.9778
8_recall: 1.0000
8_f1-score: 0.9888
8_support: 44.0000
accuracy: 0.9989
macro avg_precision: 0.9835
macro avg_recall: 0.9728
macro avg_f1-score: 0.9778
macro avg_support: 31724.0000
weighted avg_precision: 0.9989
weigh


Aggregated Metrics for 2 runs:
test_loss: 0.0158
test_accuracy: 0.9944
balanced_accuracy: 0.9191
roc_auc: 0.9994
pr_auc: 0.9983
f2: 0.9944
mcc: 0.9796
0_precision: 0.9967
0_recall: 0.9998
0_f1-score: 0.9982
0_support: 26862.0000
1_precision: 0.9964
1_recall: 0.9853
1_f1-score: 0.9908
1_support: 2657.0000
2_precision: 0.9881
2_recall: 0.9714
2_f1-score: 0.9796
2_support: 908.0000
3_precision: 0.9555
3_recall: 0.9626
3_f1-score: 0.9590
3_support: 522.0000
4_precision: 0.9695
4_recall: 0.9403
4_f1-score: 0.9545
4_support: 293.0000
5_precision: 0.9029
5_recall: 0.8494
5_f1-score: 0.8744
5_support: 269.0000
6_precision: 0.9679
6_recall: 0.9181
6_f1-score: 0.9419
6_support: 116.0000
7_precision: 0.8544
7_recall: 0.7358
7_f1-score: 0.7905
7_support: 53.0000
8_precision: 0.9389
8_recall: 0.9091
8_f1-score: 0.9230
8_support: 44.0000
accuracy: 0.9944
macro avg_precision: 0.9523
macro avg_recall: 0.9191
macro avg_f1-score: 0.9347
macro avg_support: 31724.0000
weighted avg_precision: 0.9943
weigh


Aggregated Metrics for 3 runs:
test_loss: 0.0206
test_accuracy: 0.9932
balanced_accuracy: 0.9072
roc_auc: 0.9990
pr_auc: 0.9979
f2: 0.9931
mcc: 0.9749
0_precision: 0.9960
0_recall: 0.9998
0_f1-score: 0.9979
0_support: 26862.0000
1_precision: 0.9971
1_recall: 0.9802
1_f1-score: 0.9885
1_support: 2657.0000
2_precision: 0.9809
2_recall: 0.9655
2_f1-score: 0.9731
2_support: 908.0000
3_precision: 0.9586
3_recall: 0.9393
3_f1-score: 0.9485
3_support: 522.0000
4_precision: 0.9554
4_recall: 0.9317
4_f1-score: 0.9433
4_support: 293.0000
5_precision: 0.8780
5_recall: 0.8352
5_f1-score: 0.8554
5_support: 269.0000
6_precision: 0.9488
6_recall: 0.9052
6_f1-score: 0.9262
6_support: 116.0000
7_precision: 0.7773
7_recall: 0.7296
7_f1-score: 0.7492
7_support: 53.0000
8_precision: 0.9503
8_recall: 0.8788
8_f1-score: 0.9116
8_support: 44.0000
accuracy: 0.9932
macro avg_precision: 0.9380
macro avg_recall: 0.9072
macro avg_f1-score: 0.9215
macro avg_support: 31724.0000
weighted avg_precision: 0.9931
weigh


Aggregated Metrics for 4 runs:
test_loss: 0.0245
test_accuracy: 0.9918
balanced_accuracy: 0.8911
roc_auc: 0.9988
pr_auc: 0.9975
f2: 0.9918
mcc: 0.9700
0_precision: 0.9954
0_recall: 0.9995
0_f1-score: 0.9974
0_support: 26862.0000
1_precision: 0.9916
1_recall: 0.9786
1_f1-score: 0.9850
1_support: 2657.0000
2_precision: 0.9786
2_recall: 0.9604
2_f1-score: 0.9693
2_support: 908.0000
3_precision: 0.9501
3_recall: 0.9219
3_f1-score: 0.9354
3_support: 522.0000
4_precision: 0.9626
4_recall: 0.9070
4_f1-score: 0.9330
4_support: 293.0000
5_precision: 0.8599
5_recall: 0.8151
5_f1-score: 0.8364
5_support: 269.0000
6_precision: 0.9217
6_recall: 0.8944
6_f1-score: 0.9074
6_support: 116.0000
7_precision: 0.7672
7_recall: 0.6792
7_f1-score: 0.7158
7_support: 53.0000
8_precision: 0.9435
8_recall: 0.8636
8_f1-score: 0.9006
8_support: 44.0000
accuracy: 0.9918
macro avg_precision: 0.9301
macro avg_recall: 0.8911
macro avg_f1-score: 0.9089
macro avg_support: 31724.0000
weighted avg_precision: 0.9917
weigh

...


Aggregated Metrics for 11 runs:
test_loss: 0.0308
test_accuracy: 0.9907
balanced_accuracy: 0.8810
roc_auc: 0.9985
pr_auc: 0.9968
f2: 0.9907
mcc: 0.9661
0_precision: 0.9959
0_recall: 0.9992
0_f1-score: 0.9976
0_support: 26862.0000
1_precision: 0.9853
1_recall: 0.9744
1_f1-score: 0.9798
1_support: 2657.0000
2_precision: 0.9755
2_recall: 0.9638
2_f1-score: 0.9695
2_support: 908.0000
3_precision: 0.9400
3_recall: 0.9006
3_f1-score: 0.9193
3_support: 522.0000
4_precision: 0.9709
4_recall: 0.8861
4_f1-score: 0.9260
4_support: 293.0000
5_precision: 0.8113
5_recall: 0.8253
5_f1-score: 0.8170
5_support: 269.0000
6_precision: 0.8786
6_recall: 0.8887
6_f1-score: 0.8826
6_support: 116.0000
7_precision: 0.6994
7_recall: 0.6484
7_f1-score: 0.6685
7_support: 53.0000
8_precision: 0.9455
8_recall: 0.8430
8_f1-score: 0.8895
8_support: 44.0000
accuracy: 0.9907
macro avg_precision: 0.9114
macro avg_recall: 0.8810
macro avg_f1-score: 0.8944
macro avg_support: 31724.0000
weighted avg_precision: 0.9908
weig


Aggregated Metrics for 12 runs:
test_loss: 0.0310
test_accuracy: 0.9907
balanced_accuracy: 0.8809
roc_auc: 0.9985
pr_auc: 0.9968
f2: 0.9906
mcc: 0.9658
0_precision: 0.9959
0_recall: 0.9992
0_f1-score: 0.9975
0_support: 26862.0000
1_precision: 0.9863
1_recall: 0.9729
1_f1-score: 0.9795
1_support: 2657.0000
2_precision: 0.9753
2_recall: 0.9649
2_f1-score: 0.9701
2_support: 908.0000
3_precision: 0.9395
3_recall: 0.8985
3_f1-score: 0.9180
3_support: 522.0000
4_precision: 0.9668
4_recall: 0.8896
4_f1-score: 0.9260
4_support: 293.0000
5_precision: 0.8076
5_recall: 0.8237
5_f1-score: 0.8145
5_support: 269.0000
6_precision: 0.8786
6_recall: 0.8872
6_f1-score: 0.8820
6_support: 116.0000
7_precision: 0.6961
7_recall: 0.6494
7_f1-score: 0.6679
7_support: 53.0000
8_precision: 0.9419
8_recall: 0.8428
8_f1-score: 0.8880
8_support: 44.0000
accuracy: 0.9907
macro avg_precision: 0.9098
macro avg_recall: 0.8809
macro avg_f1-score: 0.8937
macro avg_support: 31724.0000
weighted avg_precision: 0.9907
weig


Aggregated Metrics for 13 runs:
test_loss: 0.0312
test_accuracy: 0.9906
balanced_accuracy: 0.8787
roc_auc: 0.9985
pr_auc: 0.9968
f2: 0.9906
mcc: 0.9657
0_precision: 0.9958
0_recall: 0.9992
0_f1-score: 0.9975
0_support: 26862.0000
1_precision: 0.9865
1_recall: 0.9729
1_f1-score: 0.9796
1_support: 2657.0000
2_precision: 0.9750
2_recall: 0.9654
2_f1-score: 0.9701
2_support: 908.0000
3_precision: 0.9386
3_recall: 0.8995
3_f1-score: 0.9182
3_support: 522.0000
4_precision: 0.9667
4_recall: 0.8861
4_f1-score: 0.9239
4_support: 293.0000
5_precision: 0.8083
5_recall: 0.8216
5_f1-score: 0.8138
5_support: 269.0000
6_precision: 0.8781
6_recall: 0.8873
6_f1-score: 0.8818
6_support: 116.0000
7_precision: 0.6992
7_recall: 0.6357
7_f1-score: 0.6607
7_support: 53.0000
8_precision: 0.9423
8_recall: 0.8409
8_f1-score: 0.8872
8_support: 44.0000
accuracy: 0.9906
macro avg_precision: 0.9101
macro avg_recall: 0.8787
macro avg_f1-score: 0.8925
macro avg_support: 31724.0000
weighted avg_precision: 0.9906
weig


Aggregated Metrics for 14 runs:
test_loss: 0.0317
test_accuracy: 0.9906
balanced_accuracy: 0.8787
roc_auc: 0.9985
pr_auc: 0.9968
f2: 0.9905
mcc: 0.9654
0_precision: 0.9957
0_recall: 0.9993
0_f1-score: 0.9975
0_support: 26862.0000
1_precision: 0.9867
1_recall: 0.9726
1_f1-score: 0.9796
1_support: 2657.0000
2_precision: 0.9747
2_recall: 0.9640
2_f1-score: 0.9693
2_support: 908.0000
3_precision: 0.9384
3_recall: 0.8987
3_f1-score: 0.9177
3_support: 522.0000
4_precision: 0.9667
4_recall: 0.8871
4_f1-score: 0.9246
4_support: 293.0000
5_precision: 0.8088
5_recall: 0.8192
5_f1-score: 0.8129
5_support: 269.0000
6_precision: 0.8814
6_recall: 0.8836
6_f1-score: 0.8816
6_support: 116.0000
7_precision: 0.6841
7_recall: 0.6442
7_f1-score: 0.6558
7_support: 53.0000
8_precision: 0.9427
8_recall: 0.8393
8_f1-score: 0.8865
8_support: 44.0000
accuracy: 0.9906
macro avg_precision: 0.9088
macro avg_recall: 0.8787
macro avg_f1-score: 0.8917
macro avg_support: 31724.0000
weighted avg_precision: 0.9906
weig


Aggregated Metrics for 15 runs:
test_loss: 0.0323
test_accuracy: 0.9904
balanced_accuracy: 0.8763
roc_auc: 0.9985
pr_auc: 0.9967
f2: 0.9903
mcc: 0.9646
0_precision: 0.9954
0_recall: 0.9993
0_f1-score: 0.9974
0_support: 26862.0000
1_precision: 0.9873
1_recall: 0.9720
1_f1-score: 0.9796
1_support: 2657.0000
2_precision: 0.9747
2_recall: 0.9633
2_f1-score: 0.9689
2_support: 908.0000
3_precision: 0.9365
3_recall: 0.8980
3_f1-score: 0.9164
3_support: 522.0000
4_precision: 0.9680
4_recall: 0.8771
4_f1-score: 0.9192
4_support: 293.0000
5_precision: 0.8077
5_recall: 0.8146
5_f1-score: 0.8101
5_support: 269.0000
6_precision: 0.8878
6_recall: 0.8747
6_f1-score: 0.8794
6_support: 116.0000
7_precision: 0.6909
7_recall: 0.6478
7_f1-score: 0.6614
7_support: 53.0000
8_precision: 0.9448
8_recall: 0.8394
8_f1-score: 0.8876
8_support: 44.0000
accuracy: 0.9904
macro avg_precision: 0.9103
macro avg_recall: 0.8763
macro avg_f1-score: 0.8911
macro avg_support: 31724.0000
weighted avg_precision: 0.9904
weig